In [1]:
import pandas as pd
import numpy as np

In [2]:
import configparser

config = configparser.RawConfigParser()
config.read('parametros.config')
PARAMETROS_GERAIS = dict(config.items('Geral'))

In [3]:
PARAMETROS_GERAIS

{'pasta_raiz_dados': '/opt/DADOS_ANS5/',
 'arquivo_operadoras': 'http://ftp.dadosabertos.ans.gov.br/FTP/PDA/operadoras_de_plano_de_saude_ativas/Relatorio_cadop.csv',
 'arquivo_final_todas_demonstracoes': 'todas_demonstracoes_concatenadas.csv',
 'ano_inicio_analises': '2013'}

In [4]:
ARQUIVO_TODAS_DEMONSTRACOES = PARAMETROS_GERAIS['pasta_raiz_dados']+PARAMETROS_GERAIS['arquivo_final_todas_demonstracoes']

In [5]:
ARQUIVO_TODAS_DEMONSTRACOES

'/opt/DADOS_ANS5/todas_demonstracoes_concatenadas.csv'

In [6]:
df_temp = pd.read_csv(ARQUIVO_TODAS_DEMONSTRACOES,encoding="ISO-8859-1")

In [9]:
df_temp['PERIODO'].unique()

array([201501, 201502, 201503, 201504, 201301, 201302, 201303, 201304,
       201901, 201902, 201903, 201401, 201402, 201403, 201404, 201701,
       201702, 201703, 201704, 201601, 201602, 201603, 201604, 201801,
       201802, 201803, 201804])

In [10]:
df_temp = df_temp.sort_values(by="PERIODO")
df_temp.reset_index(drop=True,inplace=True)

In [11]:
lista_periodos = df_temp['PERIODO'].unique().tolist()

In [12]:
def calcular_1(df):
    serie = df['CONTRAPRESTACAO_PRE']+df['CORRESP_CEDIDA_PRE']
    return "{:.2f}".format(serie.values[0])

In [13]:
def calcular_2(df):
    serie = df['CONTRAPRESTACAO_POS']+df['CORRESP_CEDIDA_POS']
    return "{:.2f}".format(serie.values[0])

In [14]:
def calcular_3(df):
    serie = df['EVENTOS_INDENIZAVEIS_PRE']+df['VARIACAO_PEONA']
    return "{:.2f}".format(serie.values[0])

In [15]:
def calcular_4(df):
    serie = df['EVENTOS_INDENIZAVEIS_POS']
    return "{:.2f}".format(serie.values[0])

In [36]:
lista_somatorios = []
for reg_ans in df_temp['REG_ANS'].unique():
    df_operadora = df_temp[df_temp['REG_ANS'] == reg_ans].copy()
    df_operadora.reset_index(inplace=True,drop=True)
    for indice in df_operadora.index.tolist()[3:]:
#         dict_periodo = {'REG_ANS':reg_ans, 'periodo':atual};
        atual = df_operadora.iloc[indice]
        dict_periodo = {'REG_ANS':reg_ans, 'periodo':str(atual['TRIMESTRE'])+"T"+str(atual['ANO'])};
#         dict_periodo['periodo'] = str(atual['TRIMESTRE'])+"T"+str(atual['ANO'])
        _range = np.arange(indice-3,indice+1)
        df_conta = df_operadora.iloc[_range].groupby(['REG_ANS']).sum()
        dict_periodo['contas'] = {'1': calcular_1(df_conta)}
        dict_periodo['contas']['2'] = calcular_2(df_conta)
        dict_periodo['contas']['3'] = calcular_3(df_conta)
        dict_periodo['contas']['4'] = calcular_4(df_conta)
#         print(dict_periodo)
        lista_somatorios.append(dict_periodo)

In [37]:
newlist = sorted(lista_somatorios, key=lambda k: k['REG_ANS']) 

In [38]:
df_final = pd.DataFrame(newlist)

In [39]:
def gerar_colunas(x):
    x['1'] = x['contas']['1']
    x['2'] = x['contas']['2']
    x['3'] = x['contas']['3']
    x['4'] = x['contas']['4']
    return x

In [40]:
df_final = df_final.apply(lambda x: gerar_colunas(x),axis=1)

In [41]:
df_final.drop(columns="contas",inplace=True)
df_final.drop_duplicates(inplace=True)

In [42]:
df_final.to_csv("calculos_natalia.csv",index=False, encoding="ISO-8859-1")

In [43]:
len(df_final)

6660